In [62]:
import json
from collections import defaultdict
from typing import Iterable

import numpy
import pandas
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold

In [63]:
# Copy-pasted from the game.
lib = {
  "LIB_HERO_NAME_0": "Карен",
  "LIB_HERO_NAME_1": "Аврора",
  "LIB_HERO_NAME_10": "Безликий",
  "LIB_HERO_NAME_11": "Чабба",
  "LIB_HERO_NAME_12": "Арахна",
  "LIB_HERO_NAME_13": "Орион",
  "LIB_HERO_NAME_14": "Фокс",
  "LIB_HERO_NAME_15": "Джинджер",
  "LIB_HERO_NAME_16": "Данте",
  "LIB_HERO_NAME_17": "Моджо",
  "LIB_HERO_NAME_18": "Судья",
  "LIB_HERO_NAME_19": "Темная Звезда",
  "LIB_HERO_NAME_2": "Галахад",
  "LIB_HERO_NAME_20": "Артемис",
  "LIB_HERO_NAME_21": "Маркус",
  "LIB_HERO_NAME_22": "Пеппи",
  "LIB_HERO_NAME_23": "Лиэн",
  "LIB_HERO_NAME_24": "Тесак",
  "LIB_HERO_NAME_25": "Исмаил",
  "LIB_HERO_NAME_26": "Лилит",
  "LIB_HERO_NAME_27": "Лютер",
  "LIB_HERO_NAME_28": "Цин Мао",
  "LIB_HERO_NAME_29": "Дориан",
  "LIB_HERO_NAME_3": "Кира",
  "LIB_HERO_NAME_30": "Корнелиус",
  "LIB_HERO_NAME_31": "Джет",
  "LIB_HERO_NAME_32": "Гелиос",
  "LIB_HERO_NAME_33": "Ларс",
  "LIB_HERO_NAME_34": "Криста",
  "LIB_HERO_NAME_35": "Йорген",
  "LIB_HERO_NAME_36": "Майя",
  "LIB_HERO_NAME_37": "Джу",
  "LIB_HERO_NAME_38": "Эльмир",
  "LIB_HERO_NAME_39": "Зири",
  "LIB_HERO_NAME_4": "Астарот",
  "LIB_HERO_NAME_40": "Небула",
  "LIB_HERO_NAME_5": "Кай",
  "LIB_HERO_NAME_6": "Фобос",
  "LIB_HERO_NAME_7": "Тея",
  "LIB_HERO_NAME_8": "Сорвиголова",
  "LIB_HERO_NAME_9": "Хайди",
}

In [64]:
def parse_heroes(heroes: Iterable[dict], sign: int) -> dict:
    return {
        f'''{lib[f"LIB_HERO_NAME_{hero['id']}"]} {hero_key.capitalize()}''': hero[hero_key]
        for hero_key in ('level', 'color', 'star')
    }

def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    for battle_key, sign in (('player', +1), ('enemies', -1)):
        for hero in battle[battle_key]:
            for hero_key in ('Level', 'Color', 'Star'):
                result[f'''{lib[f"LIB_HERO_NAME_{hero['id']}"]} {hero_key}'''] += sign * hero[hero_key.lower()]
        
    return {'Win': battle['win'], **result}

In [65]:
def invert_column(series: pandas.Series):
    """
    Inverts the column to make an "opposite" battle.
    """
    return series == False if series.name == 'Win' else -series

battles = pandas.DataFrame([parse_battle(line) for line in open('battles.jsonl')]).fillna(value=0)
battles = pandas.concat((battles, battles.apply(invert_column)))
battles.head()

,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
0,False,0.0,0.0,0.0,4.0,35.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.0,0.0,0.0,-2.0,-10.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.0,0.0,0.0,4.0,36.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-45.0,-2.0
3,False,0.0,0.0,0.0,-1.0,-10.0,-1.0,-4.0,-46.0,-3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.0,0.0,0.0,-1.0,-4.0,-2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
battles.describe()

,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,Астарот Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
count,260.000000,260.000000,260.00000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,...,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000,260.00000,260.000000,260.00000
mean,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
std,2.555938,20.311664,0.90045,3.533622,28.288912,1.737615,3.511701,29.501521,1.252025,5.067879,...,0.215249,1.121912,9.243593,0.316838,1.459888,11.611803,0.645746,0.97458,8.172843,0.41217
min,-8.000000,-61.000000,-3.00000,-8.000000,-70.000000,-6.000000,-8.000000,-69.000000,-3.000000,-8.000000,...,-2.000000,-7.000000,-60.000000,-2.000000,-8.000000,-61.000000,-3.000000,-7.00000,-60.000000,-3.00000
25%,-0.000000,-0.000000,-0.00000,-1.000000,-5.250000,-1.000000,-0.000000,-0.000000,-0.000000,-6.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.00000,-0.000000,-0.00000
50%,0.000000,0.000000,0.00000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
75%,0.000000,0.000000,0.00000,1.000000,5.250000,1.000000,0.000000,0.000000,0.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
max,8.000000,61.000000,3.00000,8.000000,70.000000,6.000000,8.000000,69.000000,3.000000,8.000000,...,2.000000,7.000000,60.000000,2.000000,8.000000,61.000000,3.000000,7.00000,60.000000,3.00000


In [69]:
x = battles.drop(['Win'], axis=1)
y = battles['Win']

In [87]:
grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000), {
        'C': numpy.logspace(-6, 2, num=100),
    },
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    scoring='accuracy',
).fit(x, y)

In [96]:
print(f'Score: {grid_search.best_score_}')
print(f'Params: {grid_search.best_params_}')
print(f'Classes: {grid_search.best_estimator_.classes_}')
print(f'Intercept: {grid_search.best_estimator_.intercept_}')

Score: 0.9538461538461539
Params: {'C': 68.926121043497091}
Classes: [False  True]
Intercept: [ -3.48382505e-09]


In [102]:
pandas.DataFrame({'Feature': x.columns, 'Importance': grid_search.best_estimator_.coef_[0]}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False)
battles.sort_values()

,Importance
Feature,
Астарот Star,10.355195
Судья Color,7.094838
Фобос Color,6.029266
Сорвиголова Color,5.722473
Джинджер Color,5.578741
Сорвиголова Star,5.572055
Арахна Color,5.561617
Исмаил Star,5.064659
Галахад Star,4.873263


In [72]:
result = pandas.concat((
    pandas.Series(grid_search.best_estimator_.predict(x), index=battles.index, name='Predicted'),
    pandas.Series(grid_search.best_estimator_.predict_proba(x)[:, 1], index=battles.index, name='Probability'),
    battles,
), axis=1)
result['Probability'] = result['Probability'].apply('{:.2f}'.format)
result

,Predicted,Probability,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
0,False,0.00,False,0.0,0.0,0.0,4.0,35.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.00,False,0.0,0.0,0.0,-2.0,-10.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.00,False,0.0,0.0,0.0,4.0,36.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-45.0,-2.0
3,False,0.04,False,0.0,0.0,0.0,-1.0,-10.0,-1.0,-4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.00,False,0.0,0.0,0.0,-1.0,-4.0,-2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,False,0.00,False,0.0,0.0,0.0,-3.0,-13.0,0.0,-7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,False,0.00,False,0.0,0.0,0.0,-1.0,-5.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,False,0.00,False,0.0,0.0,0.0,4.0,37.0,2.0,-5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,True,0.90,True,0.0,0.0,0.0,4.0,37.0,2.0,-4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,False,0.00,False,0.0,0.0,0.0,4.0,37.0,2.0,-5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
